# MICCoM School 2017 Ex#2 : Computation of the dielectric screening

We are going to dive into the calculation of the dielectric screening done in Exercise #1.

In this excercixe we will focus on the input for `wstat.x`.

In [ ]:
# download the pseudopotentials
!wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/Si_ONCV_PBE-1.2.upf
!wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/H_ONCV_PBE-1.2.upf

# download the input files
!wget -N -q http://www.west-code.org/doc/training/silane/pw.in
!wget -N -q http://www.west-code.org/doc/training/silane/wstat.in

In order to compute the dielectric screening we need to read the output of a DFT calculation, therefore as first step we run the DFT calculation invoking the executable `pw.x` on 8 CPU cores.

In [ ]:
!mpirun -n 8 pw.x -i pw.in > pw.out

## 2.1 : Iterative diagonalization of the static dielectric matrix

Wstat will read the output of DFT and will compute the static dielectric screening from first principles. An iterative diagonalization of the dielectric matrix is performed:

\begin{equation}
\epsilon_{\mathbf{G},\mathbf{G^\prime}}(\omega=0) = \sum_{i=1}^{\texttt{n_pdep_eigen}} \, \phi_i(\mathbf{G}) \, \lambda_i \phi^\ast_i(\mathbf{G^\prime})
\end{equation}

where $\phi_i$ and $\lambda_i$ are the eigenvectors and eigenvalues of the dielectric matrix (extressed here in terms of plane waves).

The diagonalization is started with random potentials and then iterated using the Davidson algorithm. Details about the implementation are described in <a href="https://pubs.acs.org/doi/full/10.1021/ct500958p#showFigures">J. Chem. Theory Comput. 11, 2680 (2015)</a> :
<img src="https://pubs.acs.org/cms/10.1021/ct500958p/asset/images/medium/ct-2014-00958p_0003.gif" width="50%">

Let's give a quick look at the input for `wstat.x` (description of the input variables for `wstat.x` can be found here: http://www.west-code.org/doc/West/latest/manual.html#wstat-control))

In [ ]:
!cat wstat.in

In this case we are computing 50 eigenpotentials $\phi_i$.

We run `wstat.x` on 8 cores.

In [ ]:
!mpirun -n 8 wstat.x -i wstat.in > wstat.out

We now load the output (JSON) of the `wstat.x` calculation, which can be found in the `wstat.save` folder.

In [ ]:
import json

# read data from JSON file
with open('silane.wstat.save/wstat.json') as file:
    data = json.load(file)

# pretty print the data
print(json.dumps(data, indent=2))

We plot the $50$ eigenvalues $\lambda_i$ of the dielectric matrix.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# y : eigenvalues of the dielectic matrix
y = data['output']['eigenval']
# x : index of eigenvalue
x = np.arange(1, len(y)+1, 1)

print('y : ', y)
print('x : ', x)

# plot
fig, ax = plt.subplots(1, 1)
ax.plot(x, y, 'ro-'')

plt.xlabel('i')
plt.ylabel('Eigenvalue')

plt.title('PDEP')

plt.show()

We can track how the eigenvalues converge during the iterative diagonalization.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# y : eigenvalues of the dielectic matrix (now for every iteration)
y = []
for i in range(data['exec']['ndav']) :
    y.append(data['exec']['davitr'][i]['ev'])
# x : index of the eigenvalues
x = np.arange(1, len(y[0])+1, 1)

# plot
fig, ax = plt.subplots(1, 1)

for i in range(data['exec']['ndav']) :
    ax.plot(x, y[i], 'o-', label='iter'+str(i+1))

plt.xlabel('i')
plt.ylabel('Eigenvalue')

plt.legend()

plt.title('PDEP')
plt.show()

<b>Question</b>: <i>How do we compute 70 eigenpotentials?</i>

<i>~ Edit the file below, execute it, and plot the eigenvalues! ~</i>

In [ ]:
import yaml

# read data: wstat.in --> input_data
with open('wstat.in') as file:
    input_data = yaml.load(file, Loader=yaml.FullLoader)

# pretty print
print('input_data')
print(yaml.dump(input_data, sort_keys=False))

# modify the variable n_pdep_eigen in input_data
# input_data[..........  EDIT HERE ...............] = 70

# write data to YAML file: input_data --> wstat_70.in
with open('wstat_70.in', 'w') as file:
    yaml.dump(input_data, file, sort_keys=False)

# show content of the file
print('wstat_70.in')
!cat wstat_70.in

In [ ]:
!mpirun -n 8 wstat.x -i wstat_70.in > wstat_70.out

Now plot the data and compare.